# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?

In [ ]:
How oxycodone withdrawal affects gene expression in brain reward circuits (NAc, mPFC, VTA) under conditions of chronic neuropathic pain.

What do the conditions mean?

Difference treatments the mices got:


oxy: Oxicodone (Reference Group)

sal: Saline (Control Group)

What do the genotypes mean?

SNI:
spared nerve injury (SNI) (Chronic Pain)

Sham:
sham controls - also surgery but not a nerve injury (None Chronic Pain)

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?
Which groups would you compare to each other?

I would try to find out if there exist any relations between the transcriptome related to the treatments and the genotypes. Afterwards, I would follow up with identifying the most relevant ones.

Please also mention which outcome you would expect to see from each comparison.

I expect the most transcriptomical change in the case of oxy-SNI compared to all other groups. The brain should get be used to the beneficial pain reducing effect of oxy. 

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [24]:
import pandas as pd
import numpy as np

df = pd.read_excel("conditions_runs_oxy_project.xlsx", index_col="Run")
# Show first rows

df = df.fillna(False)
df = df.replace("x", True)

df['condition'] = np.select(
    condlist=[
        df["condition: Sal"] == 1,
        df["Condition: Oxy"] == 1
    ],
    choicelist=["Sal", "Oxy"],
    default=None
)
    
df['geno_type'] = np.select(
    condlist=[
        df["Genotype: SNI"] == 1,
        df["Genotype: Sham"] == 1
    ],
    choicelist=["SNI", "Sham"],
    default=None
)

df = df.drop(columns=["condition: Sal", "Condition: Oxy", "Genotype: SNI", "Genotype: Sham"])

df.head()

/var/folders/ml/pqmgm7zn4257t4bl2n8dv9d80000gn/T/ipykernel_2757/3668940756.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/var/folders/ml/pqmgm7zn4257t4bl2n8dv9d80000gn/T/ipykernel_2757/3668940756.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("x", True)


,Patient,RNA-seq,DNA-seq,condition,geno_type
Run,,,,,
SRR23195505,?,True,False,Sal,SNI
SRR23195506,?,True,False,Oxy,Sham
SRR23195507,?,True,False,Sal,Sham
SRR23195508,?,True,False,Oxy,SNI
SRR23195509,?,True,False,Oxy,SNI


In [25]:
df['condition'].value_counts()

condition
Sal    8
Oxy    8
Name: count, dtype: int64

In [26]:
df['geno_type'].value_counts()

geno_type
SNI     8
Sham    8
Name: count, dtype: int64

In [27]:
df.groupby(['condition', 'geno_type']).size()

condition  geno_type
Oxy        SNI          4
           Sham         4
Sal        SNI          4
           Sham         4
dtype: int64

They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [34]:
base_counts = pd.read_csv("base_counts.csv", names=["Run", "Bases"])
base_counts = base_counts.set_index("Run")
base_counts = base_counts.sort_values("Bases", ascending=True)
base_counts

,Bases
Run,
SRR23195516,6203117700
SRR23195511,6456390900
SRR23195517,6863840400
SRR23195505,6922564500
SRR23195508,6927786900
SRR23195519,6996050100
SRR23195509,7003550100
SRR23195514,7226808600
SRR23195510,7377388500


In [36]:
base_counts[:2]

,Bases
Run,
SRR23195516,6203117700
SRR23195511,6456390900


In [ ]:
!nextflow run nf-core/fetchngs \
   -profile docker \
   --input ids.csv \
   --outdir fetchngs_out \
   --max_memory "12.GB"

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.